In [1]:
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
from skimage import data, color
from PIL import Image
import cv2
from numpy import savez_compressed
from google.colab import files
from numpy import load
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from google.colab import drive
drive.mount('/content/gdrive')

filename = '/content/gdrive/My Drive/Research2/np_data/x.npz'
x = np.load(filename)['arr_0']

filename = '/content/gdrive/My Drive/Research2/np_data/y.npz'
y = np.load(filename)['arr_0']

print(x.shape)
print(y.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
(270912, 56, 56, 1)
(270912, 1)


In [2]:
baseAccuracies = []
losses = []
balancedAccuracies = []
f1scores = []
precisions = []
recalls = []

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
callback = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1)

for train, test in kfold.split(x, y):
  print("_________________________________________________________________________________________________________________________________")
  print("Fold number: " + str(fold_no))
  fold_no = fold_no+1
  print()

  xtrain = x[train]
  xtest = x[test]
  
  model = tf.keras.applications.ResNet50(include_top=True, weights=None, input_shape=(56, 56, 1), classes=49, classifier_activation="softmax",)
  model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["sparse_categorical_accuracy"])
  model.fit(xtrain, y[train], verbose = 1, validation_data = (xtest, y[test]), epochs = 10, callbacks = [callback], batch_size = 32)

  test_loss, test_accuracy = model.evaluate(xtest, y[test])
  print("_________________________________________________________________________________________________________________________________")
  print("Test Accuracy:")
  print(test_accuracy)
  baseAccuracies.append(test_accuracy)
  print("_________________________________________________________________________________________________________________________________")
  print("Test Loss:")
  print(test_loss)
  losses.append(test_loss)
  print("_________________________________________________________________________________________________________________________________")
  preds = np.argmax(model.predict(xtest), axis=-1)
  preds = preds.reshape(len(xtest), 1)
  accs = []
  for cls in range(49):
    mask = (y[test] == cls)
    cls_acc = (preds == cls)[mask].mean() 
    accs.append(cls_acc)
  accs = np.mean(accs)
  print("Final balanced accuracy:")
  print(accs)
  balancedAccuracies.append(accs)
  print("_________________________________________________________________________________________________________________________________")
  f1scores.append(f1_score(y[test], preds, average = 'weighted'))
  precisions.append(precision_score(y[test], preds, average = 'weighted'))
  recalls.append(recall_score(y[test], preds, average = 'weighted'))
  print("*********************************************************************************************************************************")
  print()
  print()
  del model, xtrain, xtest

_________________________________________________________________________________________________________________________________
Fold number: 1

Epoch 1/10
7620/7620 [==============================] - 310s 41ms/step - loss: 0.6965 - sparse_categorical_accuracy: 0.8107 - val_loss: 0.2828 - val_sparse_categorical_accuracy: 0.9203
Epoch 2/10
7620/7620 [==============================] - 309s 41ms/step - loss: 0.1935 - sparse_categorical_accuracy: 0.9457 - val_loss: 0.1833 - val_sparse_categorical_accuracy: 0.9503
Epoch 3/10
7620/7620 [==============================] - 308s 40ms/step - loss: 0.1241 - sparse_categorical_accuracy: 0.9650 - val_loss: 0.1446 - val_sparse_categorical_accuracy: 0.9606
Epoch 4/10
7620/7620 [==============================] - 308s 40ms/step - loss: 0.0931 - sparse_categorical_accuracy: 0.9735 - val_loss: 0.1140 - val_sparse_categorical_accuracy: 0.9688
Epoch 5/10
7620/7620 [==============================] - 308s 40ms/step - loss: 0.0722 - sparse_categorical_accurac

In [3]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(balancedAccuracies)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {losses[i]} - class-wise Accuracy: {balancedAccuracies[i]}% Accuracy - {baseAccuracies[i]} - Precision: {precisions[i]} - Recall: {recalls[i]} - F1: {f1scores[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(balancedAccuracies)} (+- {np.std(balancedAccuracies)})')
print(f'> Loss: {np.mean(losses)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.10471563041210175 - class-wise Accuracy: 0.9702628923010977% Accuracy - 0.9732024073600769 - Precision: 0.9741182174061239 - Recall: 0.9732024213790049 - F1: 0.973386489149957
------------------------------------------------------------------------
> Fold 2 - Loss: 0.10313615947961807 - class-wise Accuracy: 0.9747059030200309% Accuracy - 0.9753432869911194 - Precision: 0.9756897735965739 - Recall: 0.9753432747674591 - F1: 0.9753739141607072
------------------------------------------------------------------------
> Fold 3 - Loss: 0.10730736702680588 - class-wise Accuracy: 0.970916124213864% Accuracy - 0.9739766120910645 - Precision: 0.9742191938455931 - Recall: 0.9739765973939685 - F1: 0.973978250561554
------------------------------------------------------------------------
> Fold 4 - Loss: 0.0865599885582923